In [132]:
import numpy as np
import pandas as pd
import math
import os.path
import json
import time

In [133]:
DATA_ROOT_PATH = '/src/data/wi-voting'
WI_COUNTY_TURNOUT = 'WI-county.json'
WI_COUNTY_REG_VOTER = 'RegisteredVotersByCounty_11-01-2020.csv'
COUNTY_FISP = 'county_fips_master.csv'

In [134]:
reg_voter_df =  pd.read_csv(os.path.join(DATA_ROOT_PATH, WI_COUNTY_REG_VOTER), encoding='us-ascii')

In [135]:
reg_voter_df['Registered Voters'] = reg_voter_df['Registered Voters'].apply(int)
reg_voter_df.head(3)

,CountyCode,County,Registered Voters
0,1,ADAMS COUNTY,12805
1,2,ASHLAND COUNTY,9770
2,3,BARRON COUNTY,27632


In [136]:
reg_voter_df.dtypes

CountyCode            int64
County               object
Registered Voters     int64
dtype: object

In [137]:
# file -i county_fips_master.csv 
# county_fips_master.csv: application/csv; charset=iso-8859-1

fips_df =  pd.read_csv(os.path.join(DATA_ROOT_PATH, COUNTY_FISP), encoding='iso-8859-1')

In [138]:
fips_df.head(3)

,fips,county_name,state_abbr,state_name,long_name,sumlev,region,division,state,county,crosswalk,region_name,division_name
0,1001,Autauga County,AL,Alabama,Autauga County AL,50.0,3.0,6.0,1.0,1.0,3-6-1-1,South,East South Central
1,1003,Baldwin County,AL,Alabama,Baldwin County AL,50.0,3.0,6.0,1.0,3.0,3-6-1-3,South,East South Central
2,1005,Barbour County,AL,Alabama,Barbour County AL,50.0,3.0,6.0,1.0,5.0,3-6-1-5,South,East South Central


In [139]:
fips_df.dtypes

fips               int64
county_name       object
state_abbr        object
state_name        object
long_name         object
sumlev           float64
region           float64
division         float64
state            float64
county           float64
crosswalk         object
region_name       object
division_name     object
dtype: object

In [140]:
with open(os.path.join(DATA_ROOT_PATH, WI_COUNTY_TURNOUT)) as f:
    wi_county_map = json.load(f)
    turnout_map = wi_county_map['data']
    

In [141]:
print(turnout_map)

{'55001': {'unit': '55001', 'ts': '2020-11-06T00:29:38.960Z', 'pr': 20, 'tp': 20, 'tv': 11806, 'cand': [{'fname': 'Donald', 'name': 'Trump', 'party': 'gop', 'votes': 7362, 'inc': True}, {'fname': 'Joe', 'name': 'Biden', 'party': 'dem', 'votes': 4329, 'winner': 'X'}, {'fname': 'Jo', 'name': 'Jorgensen', 'party': 'lib', 'votes': 85}, {'fname': 'Don', 'name': 'Blankenship', 'party': 'cst', 'votes': 27}, {'fname': 'Brian', 'name': 'Carroll', 'party': 'asp', 'votes': 3}], 'status': 'R'}, '55003': {'unit': '55003', 'ts': '2020-11-06T00:29:38.960Z', 'pr': 17, 'tp': 17, 'tv': 8728, 'cand': [{'fname': 'Joe', 'name': 'Biden', 'party': 'dem', 'votes': 4794, 'winner': 'X'}, {'fname': 'Donald', 'name': 'Trump', 'party': 'gop', 'votes': 3845, 'inc': True}, {'fname': 'Jo', 'name': 'Jorgensen', 'party': 'lib', 'votes': 67}, {'fname': 'Don', 'name': 'Blankenship', 'party': 'cst', 'votes': 11}, {'fname': 'Brian', 'name': 'Carroll', 'party': 'asp', 'votes': 11}], 'status': 'D'}, '55005': {'unit': '55005'

In [142]:
turn_out_per_county = dict()
for county_fips, item in turnout_map.items():
    vote_count = 0
    for cand in item['cand']:
        vote_count = cand['votes'] + vote_count
    if county_fips.isdigit():
        turn_out_per_county[int(county_fips)] = vote_count

In [143]:
# county FISP -> total votes
print(turn_out_per_county)

{55001: 11806, 55003: 8728, 55005: 25316, 55007: 10876, 55009: 143623, 55011: 7798, 55013: 10138, 55015: 30797, 55017: 35908, 55019: 14866, 55021: 33847, 55023: 8695, 55025: 343599, 55027: 48415, 55029: 20076, 55031: 24595, 55033: 23526, 55035: 58069, 55037: 2939, 55039: 57193, 55041: 5050, 55043: 25333, 55045: 21351, 55047: 10644, 55049: 13961, 55051: 4031, 55053: 10179, 55055: 47860, 55057: 13683, 55059: 88467, 55061: 12066, 55063: 67609, 55065: 8547, 55067: 11156, 55069: 16496, 55071: 44477, 55073: 76603, 55075: 22959, 55077: 9050, 55078: 1585, 55079: 457318, 55081: 22560, 55083: 23211, 55085: 24111, 55087: 107956, 55089: 61244, 55091: 4138, 55093: 23260, 55095: 26318, 55097: 40510, 55099: 8642, 55101: 106148, 55103: 9005, 55105: 85150, 55107: 7879, 55109: 56542, 55111: 36172, 55113: 10463, 55115: 22878, 55117: 65855, 55119: 10678, 55121: 15341, 55123: 15907, 55125: 15338, 55127: 57441, 55129: 10353, 55131: 88026, 55133: 267159, 55135: 29081, 55137: 13564, 55139: 93790, 55141: 41196

In [144]:
turn_out_per_county_df = pd.DataFrame(turn_out_per_county.items(), columns=['fips', 'votes'])

In [145]:
turn_out_per_county_df.head(3)

,fips,votes
0,55001,11806
1,55003,8728
2,55005,25316


In [146]:
turn_out_per_county_df = turn_out_per_county_df.merge(fips_df, on='fips', how='left')

In [147]:
turn_out_per_county_df['County'] = turn_out_per_county_df['county_name'].str.upper()

In [148]:
turn_out_per_county_df.head(3)

,fips,votes,county_name,state_abbr,state_name,long_name,sumlev,region,division,state,county,crosswalk,region_name,division_name,County
0,55001,11806,Adams County,WI,Wisconsin,Adams County WI,50.0,2.0,3.0,55.0,1.0,2-3-55-1,Midwest,East North Central,ADAMS COUNTY
1,55003,8728,Ashland County,WI,Wisconsin,Ashland County WI,50.0,2.0,3.0,55.0,3.0,2-3-55-3,Midwest,East North Central,ASHLAND COUNTY
2,55005,25316,Barron County,WI,Wisconsin,Barron County WI,50.0,2.0,3.0,55.0,5.0,2-3-55-5,Midwest,East North Central,BARRON COUNTY


In [149]:
turn_out_per_county_df = turn_out_per_county_df.merge(reg_voter_df, on='County', how='left')

In [150]:
turn_out_per_county_df.head(3)

,fips,votes,county_name,state_abbr,state_name,long_name,sumlev,region,division,state,county,crosswalk,region_name,division_name,County,CountyCode,Registered Voters
0,55001,11806,Adams County,WI,Wisconsin,Adams County WI,50.0,2.0,3.0,55.0,1.0,2-3-55-1,Midwest,East North Central,ADAMS COUNTY,1,12805
1,55003,8728,Ashland County,WI,Wisconsin,Ashland County WI,50.0,2.0,3.0,55.0,3.0,2-3-55-3,Midwest,East North Central,ASHLAND COUNTY,2,9770
2,55005,25316,Barron County,WI,Wisconsin,Barron County WI,50.0,2.0,3.0,55.0,5.0,2-3-55-5,Midwest,East North Central,BARRON COUNTY,3,27632


In [151]:
turn_out_per_county_df.dtypes

fips                   int64
votes                  int64
county_name           object
state_abbr            object
state_name            object
long_name             object
sumlev               float64
region               float64
division             float64
state                float64
county               float64
crosswalk             object
region_name           object
division_name         object
County                object
CountyCode             int64
Registered Voters      int64
dtype: object

In [152]:
turn_out_per_county_df['turn_out'] = turn_out_per_county_df['votes'] / turn_out_per_county_df['Registered Voters']

In [153]:
pd.set_option('display.max_rows', None)
print(turn_out_per_county_df[['county_name', 'votes', 'Registered Voters', 'turn_out']].sort_values('turn_out', ascending= False))

           county_name   votes  Registered Voters  turn_out
50        Price County    8642               9055  0.954390
60       Taylor County   10678              11294  0.945458
38    Marquette County    9050               9579  0.944775
28       Juneau County   13683              14567  0.939315
6       Burnett County   10138              10795  0.939138
54         Rusk County    7879               8396  0.938423
69     Waushara County   13564              14465  0.937712
42       Oconto County   23211              24759  0.937477
46        Pepin County    4138               4414  0.937472
58      Shawano County   22878              24408  0.937316
30     Kewaunee County   12066              12926  0.933467
7       Calumet County   30797              33050  0.931831
66   Washington County   88026              94481  0.931679
19  Fond du Lac County   57193              61390  0.931634
5       Buffalo County    7798               8405  0.927781
32    Lafayette County    8547          